In [1]:
import pandas as pd
import paddlehub as hub

In [2]:
data = pd.read_csv('../data/data_all.csv')

In [ ]:
demo_set = data.loc[0:49, :]
demo_set

In [ ]:
# paddlenlp：模型需要有标签数据微调，没用
# import paddlenlp
# import paddle
#
# MODEL_NAME_1 = 'ernie-1.0-base-zh'
# MODEL_NAME_2 = 'ernie-3.0-base-zh'
# ernie1_model = paddlenlp.transformers.ErnieModel.from_pretrained(MODEL_NAME_1)
# tokenizer = paddlenlp.transformers.ErnieTokenizer.from_pretrained(MODEL_NAME_1)
# encoded_text = tokenizer(text=demo_set['comment'])
# input_ids = paddle.to_tensor([encoded_text['input_ids']])

# 筛选某天数据
# data[data['date'] == '2022-06-10']
# data[data['date'] == '2022-06-16']

# paddlehub
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' 
module = hub.Module(name="ernie_skep_sentiment_analysis")
sentiment = module.predict_sentiment(demo_set['comment'], use_gpu=True)
sentiment

In [ ]:
sentiment_df = pd.DataFrame(sentiment)
sentiment_df

In [ ]:
demo_set_sentiment = pd.concat([demo_set, sentiment_df], axis=1)
demo_set_sentiment

In [ ]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment')
model = BertForSequenceClassification.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment').to('cuda')

In [ ]:
import numpy as np
comment = np.array(demo_set['comment']).tolist()
text = '当有知识见识，有人生理解的人（还是老师）来带货，我只能说，降维打击。懂得都懂好叭'
#tokenizer(comment, padding=True)['input_ids']
output = model(torch.tensor([tokenizer.encode(text)]).to('cuda'))
probs = torch.nn.functional.softmax(output.logits, dim=-1)
probs

In [ ]:
probs_df = pd.DataFrame(probs.data)
probs_df

In [ ]:
test_demo = data.sample(20)
test_demo
test_demo['sentiment_label'] = test_demo['comment'].apply(lambda x: module.predict_sentiment([x], use_gpu=True)[0]['sentiment_label'])
test_demo['positive_probs'] = test_demo['comment'].apply(lambda x: module.predict_sentiment([x], use_gpu=True)[0]['positive_probs'])
test_demo['negative_probs'] = test_demo['comment'].apply(lambda x: module.predict_sentiment([x], use_gpu=True)[0]['negative_probs'])
test_demo

In [ ]:
test_demo = data.sample(20)
test_demo
test_demo['sentiment_label'] = test_demo['comment'].apply(lambda x: module.predict_sentiment([x], use_gpu=True)[0]['sentiment_label'])
test_demo['positive_probs'] = test_demo['comment'].apply(lambda x: module.predict_sentiment([x], use_gpu=True)[0]['positive_probs'])
test_demo['negative_probs'] = test_demo['comment'].apply(lambda x: module.predict_sentiment([x], use_gpu=True)[0]['negative_probs'])
test_demo